In [16]:

import os
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader,Subset
import torch.optim as optim
import json

In [7]:

class MLPClassifier(nn.Module):
    def __init__(self, input_dim=283, hidden_dims=[128, 64], num_classes=12):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dims[0]),
            nn.ReLU(),
            nn.Linear(hidden_dims[0], hidden_dims[1]),
            nn.ReLU(),
            nn.Linear(hidden_dims[1], num_classes)
        )

    def forward(self, x):
        return self.model(x)


In [8]:
def prediction(path_model,x,threshold):
    model = MLPClassifier()
    model.load_state_dict(torch.load(path_model))
    model.eval()
    with torch.no_grad():
        input = x.unsqueeze(0)
        y = model(input)
        y_probs = torch.sigmoid(y)
        y_preds = (y_probs > threshold).int()
    return y_probs,y_preds

In [9]:
root = "C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/"
inputs = []
targets = []
for dirpath, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith(".pt"):

                path = root + "/" + filename
                print(path)
                data = torch.load(path,weights_only=False)
                input_array = data["input"]
                print(np.shape(input_array))

                input = input_array.reshape(-1)
                target = data["output"]
                inputs.append(input)
                targets.append(target)


X = torch.stack(inputs)
y = torch.stack(targets)
dataset = TensorDataset(X,y)
indices = list(range(len(dataset)))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_set = Subset(dataset, train_idx)
test_set = Subset(dataset, test_idx)

train_dataloader = DataLoader(train_set, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=32, shuffle=True)


C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0705_00_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0705_15_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0705_30_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0705_45_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0706_00_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0706_15_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0706_30_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0706_45_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0707_15_00.pt
torch.Size([283])
C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset//25_07_0707_30_00.pt
torch.Size([283])


In [10]:
model = MLPClassifier()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.5e-4)
num_epochs = 50
train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()
    epoch_train_loss = 0
    for X,Y in train_dataloader:
        optimizer.zero_grad()
        output = model(X)
        loss = criterion(output, Y)
        loss.backward()
        optimizer.step()
        epoch_train_loss += loss.item()
    train_losses.append(epoch_train_loss)
    epoch_val_loss = 0
    with torch.no_grad():
        for X,Y in test_dataloader:
            output = model(X)
            epoch_val_loss += criterion(output, Y).item()
    val_losses.append(epoch_val_loss)
torch.save(model.state_dict(), "./model_0_1.pth")
print(train_losses)
print(val_losses)

[31.141155779361725, 12.282804444432259, 7.41680471599102, 6.111062854528427, 5.525396212935448, 5.145126409828663, 4.940670497715473, 4.740323595702648, 4.580525226891041, 4.451656207442284, 4.3425769582390785, 4.194373160600662, 4.034865900874138, 3.921803005039692, 3.8130132257938385, 3.6852488964796066, 3.6257490813732147, 3.5144941583275795, 3.4417519867420197, 3.334175191819668, 3.274502631276846, 3.2312470488250256, 3.167315497994423, 3.112078219652176, 3.070370115339756, 3.022739637643099, 2.973552968353033, 2.935114096850157, 2.884118940681219, 2.8632095493376255, 2.8350962810218334, 2.807839449495077, 2.7665520310401917, 2.7304173707962036, 2.70038303732872, 2.6751047000288963, 2.6648165322840214, 2.63455880433321, 2.6020909287035465, 2.598510045558214, 2.5544376373291016, 2.5237426720559597, 2.4928539618849754, 2.463333459571004, 2.450568597763777, 2.445556618273258, 2.4122881293296814, 2.381480509415269, 2.3783021084964275, 2.3323256224393845]
[4.644226998090744, 2.12077338

In [15]:
test = torch.load('C:/Users/hugom/OneDrive/Documents/Stage_2025/dev_Cnn/dataset/25_07_0706_45_00.pt',weights_only=False)
treshold = 0.5
probs,pred = prediction('model_0_1.pth',test['input'],treshold)
print(pred)
print(test['output'])

tensor([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])


In [14]:
model = MLPClassifier()
model.load_state_dict(torch.load('model_0_1.pth'))
model.eval()
input = test['input']
output = test['output']
print(output)
with torch.no_grad():
    input = input.unsqueeze(0)
    y = model(input)
    y_probs = torch.sigmoid(y)
    y_preds = (y_probs > 0.5).int()
    print(y_probs)
    print(y_preds)

tensor([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([[1.1962e-05, 2.3529e-04, 7.6281e-03, 7.2035e-01, 1.1259e-02, 5.3601e-01,
         2.5232e-04, 6.2264e-04, 4.3575e-03, 6.0978e-03, 5.2287e-03, 1.2847e-04]])
tensor([[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [14]:
# 3) Export ONNX avec dynamic_axes
torch.onnx.export(
    model,
    r"C:/Users/gabriel/Documents/Stage_2025/dev_Cnn/model_0_1.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input":  {0: "batch_size"},
        "output": {0: "batch_size"},
    },
    opset_version=13,
)
print("Exporté model.onnx avec batch dynamique")

Exporté mobilenetv2-dynamic.onnx avec batch dynamique
